<h1>SpaceX  Falcon 9 first stage Landing Prediction</h1>

---

# 4.1 Launch Sites Locations Analysis with Folium


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and some of them might be discovered by analyzing the existing locations; describing some geographical patterns about those locations.


### Objectives


- Mark all launch sites on a map
- Mark the success/failed launches for each site on the map
- Calculate the distances between a launch site to its proximities


---

In [254]:
# !pip3 install folium
# !pip3 install wget

In [255]:
import pandas as pd
import numpy as np
import random as r
import wget

import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

from math import sin, cos, sqrt, atan2, radians

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [256]:
spacex_csv = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
df = pd.read_csv(spacex_csv)

geo_cols = df[['Launch Site', 'Lat', 'Long']]
geo_df = geo_cols.groupby(['Launch Site'], as_index=False).first()

print(geo_df.shape)
geo_df

(4, 3)


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


## 1. Mark all launch sites on a map


Firstly we create a folium `Map` object, with an initial center location to be **NASA Johnson Space Center at Houston, Texas**.


In [257]:
# Start location is NASA Johnson Space Center
nasa = [29.559684888503615, -95.0830971930759]
map = folium.Map(location = nasa, zoom_start = 4.25)

''' folium.Circle to add a highlighted circle area with a text label on a specific coordinate '''
# Create circle at NASA's coordinate with a popup label showing its name
circle = folium.Circle(nasa, radius=1000, color='#02adc7', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker( nasa,
                            # Create an icon as a text label
                            icon= DivIcon( icon_size=(20,20),
                                           icon_anchor=(10,15),
                            html='<div style="font-size: 12; color:#017a8c;"><b> %s </b></div>' % 'NASA',
                                        )
                          )
map.add_child(circle)
map.add_child(marker)

for loc in range( geo_df.shape[0] ):
  name = str( geo_df['Launch Site'][loc] )
  coor = [geo_df['Lat'][loc], geo_df['Long'][loc]]

  c = folium.Circle(coor, radius=1000, color='orange', fill=True).add_child(folium.Popup(name))
  m = folium.map.Marker( coor, icon= DivIcon(icon_size = (25,25), icon_anchor = (10, 15),
                                     html = '<div style="font-size: 12; color:#cc5a02;"><b> %s </b></div>' % name.split('-')[0]
                                             )
                      )

  map.add_child(c)
  map.add_child(m)

map

At first, it becomes clear that the main common characteristic of the launch zones, marked in orange, is their proximity to the coast, apart from that they do not seem to share other obvious common characteristics such as alignment with respect to the Equator or a common angle.


## 2. Mark the success/failed launches for each site on the map


Next, we enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.



In [258]:
# Function to assign color to launch class_
def class_color(class_):
  if class_ == 1: return 'green'
  else: return 'red'

df['color'] = df['class'].apply(class_color)

In [259]:
cluster = MarkerCluster()
# Add cluster to current map
map.add_child(cluster)

for i, row in df.iterrows():
  coor = [row['Lat'], row['Long']]

  marker = folium.Marker( coor, icon = folium.Icon(color = 'white', icon_color= row['color']) )
  cluster.add_child(marker)

map

From the color-labeled markers in marker clusters, we are able to easily identify which launch sites have relatively high success rates.


## 3. Calculate the distances between a launch site to its proximities


Finally, we explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while exploring the map, one can easily find the coordinates of any points of interests; such as railway, coast line, highway, etc.


In [ ]:
# Add Mouse Position to get the coordinate for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse = MousePosition( position='topright',
                                separator=' Long: ',
                                empty_string='NaN',
                                lng_first=False,
                                num_digits = 20,
                                prefix='Lat:',
                                lat_formatter = formatter,
                                lng_formatter = formatter,
                              )
map.add_child(mouse)

We can calculate the distance between two points on the map based on their `Lat` and `Long`, we write a function for this purpose.


In [261]:
def distance(lat1, lon1, lat2, lon2):
  lat1 = radians(lat1); lon1 = radians(lon1)
  lat2 = radians(lat2); lon2 = radians(lon2)
  dlon = (lon2 - lon1); dlat = (lat2 - lat1)

  a = sin(dlat / 2)**2 + ( sin(dlon / 2)**2 * ( cos(lat1) * cos(lat2) ) )
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  R = 6373 # Radius of earth [km]
  distance = R * c

  return distance

We Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site, and some other Land marks too.


In [263]:
def get_coord(df):
  coords = []
  for i, row in df.iterrows(): coords.append( [row['Lat'], row['Long']] )
  return coords
coords = get_coord(geo_df)

coast = [28.56241, -80.56786]; d = distance(coords[0][0], coords[0][1], coast[0], coast[1])
highway = [28.56339, -80.58692]
city = [28.10521, -80.63623]


print( coords[0], coast, d, sep = '\n')

[28.56230197, -80.57735648]
[28.56241, -80.56786]
0.9278161175768164


Once the coordinates and distance are obtained, we create a `folium.PolyLine` to show the distance graphically.

In [264]:
def landmark(launch_coord, landmark_coord, rand = False):
  d = distance(launch_coord[0], launch_coord[1], landmark_coord[0], landmark_coord[1])
  if rand:
    m = folium.Marker(np.array(landmark_coord) + r.uniform(-1,1) , icon= DivIcon(icon_size=(20,20),
                                          icon_anchor=(0,0),
        html='<div style="font-size: 10; color:#01022b;"><b>%s</b></div>' % f' {round(d, 3)} '
                                          )
                    )
  else:
    m = folium.Marker(landmark_coord , icon= DivIcon(icon_size=(20,20),
                                          icon_anchor=(0,0),
        html='<div style="font-size: 10; color:#01022b;"><b>%s</b></div>' % f' {round(d, 3)} '
                                          )
                    )
  map.add_child(m)

  line_coord = [launch_coord, landmark_coord]
  line = folium.PolyLine( locations = line_coord, weigth = 1.25)
  map.add_child(line)

landmark(coords[0], coast); landmark(coords[1], highway)
landmark(coords[0], city); landmark(coords[2], nasa, True); landmark(coords[3], nasa, True)

In [265]:
map

As is natural, all the launching stations are not only close to the coast, but also far from sparsely populated urban centers and whose locations are fed by train tracks or highways that support the equipment of several tons that is normally employed.
